# Transformer
Bienvenue à l'un des TPs les plus longues de ce repo. Aujourd'hui on va s'intéresser à l'architecture des transformers.
Un transformer est un réseau de neurones qui permet de transformer des séquences de données en d'autres séquences de données. Il est très utilisé dans le domaine du NLP pour faire de la traduction de texte par exemple. C'est aussi l'architecture derrière le fameux ChatGPT, que vous connaissez déjà.

Les modèles de génération d'images comme DALLE utilisent aussi des transformers, pour encoder le texte.

![chatgpt](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/chatgpt.png)

![dalle](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/dalle.png)

## Overview

Personnellement, je trouve que c'est plus facile à comprendre les transformers en commençant par comprendre comment l'utiliser (comprendre les entrées et les sorties) et ensuite en regardant l'architecture.
On va donc commencer par un exemple d'utilisation, puis on va rentrer dans le détail de l'architecture.
Donc pour l'instant, on va considérer le transformer comme un black box.


## Inférence (ou la phase d'utilisation/prédiction)

Pendant l'inférence, le transformer prédit un token (ou un mot, ou un élément de la séquence, c'est la même chose) à la fois. Il prend en compte les tokens précédemment prédits et la séquence d'origine comme entrées pour prédire le token suivant.

Considérons l'exemple de la traduction : le transformer utilise à la fois la phrase source et les mots précédemment prédits de la phrase traduite comme entrées afin de générer le mot suivant de la phrase traduite.

![blackbox](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/blackbox.png)

Pendant la traduction, le transformer prend en entrée la phrase source et la phrase actuellement prédite, puis génère le mot suivant dans la phrase traduite en se basant sur ces deux entrées. Il répète ce processus, en prenant les mots précédemment prédits et la phrase source en entrée à chaque étape, jusqu'à ce que la phrase traduite complète soit générée.

Examinons ça avec un exemple détaillé.

Supposons que le transformer soit déjà entraîné et qu'on veut qu'il traduise "I love oranges." en français. La phrase traduite qu'on veut obtenir est "J'aime les oranges".

La phrase source est `["I", "love", "oranges", "."]`.

Comme c'est le début de l'inférence, la phrase actuellement prédite est vide, mais on ne peut pas mettre une phrase vide dans le transformer, donc on commence par `["<start>"]` comme phrase actuellement prédite.

On a donc ces deux séquences en entrées: `["I", "love", "oranges", "."]` et `["<start>"]`.

On va tokenizer ensuite ces séquences (convertir chaque mot en un nombre le représentant), par exemple `["I", "love", "oranges", "."]` -> `[10, 35, 20, 49]` et `["<start>"]` -> `[40]`.

Le transformer prend ensuite en entrée ces deux séquences, puis génère un vecteur de probabilité sur l'ensemble du vocabulaire, représentant la probabilité de chaque mot possible suivant le token `<start>`.

Puisque le transformer est déjà entraîné, le mot avec la probabilité la plus élevée va être `J'`. On ajoute `J'` à la phrase actuellement prédite.

![first inference](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/probability1.png)

On a donc maintenant `["I", "love", "oranges", "."]` et `["<start>", "J'"]`. Avec la tokenisation, on a `[10, 35, 20, 49]` et `[40, 20]`, en supposant que 20 représente `J'`.
Remarque qu'on a deux tokeniseurs différents pour l'anglais et le français.

Le transformer prend ensuite en entrée ces séquences, puis génère deux vecteurs de probabilité, la première représentant la probabilité du mot possible suivant le token `<start>`, et la seconde celle du mot possible suivant le token `J'`.

![second inference](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/probability2.png)

Remarque que le premier vecteur de probabilité est identique au précédent. C'est parce que le transformer génère toujours les même vecteurs pour les mêmes entrées même s'il y a des tokens supplémentaires dans la phrase actuellement prédite. Les futurs tokens n'influencent pas les vecteurs de probabilité des tokens précédents. C'est parce qu'il y a un masque qu'on appelle `causal mask` qui empêche les tokens de prêter attention aux futurs tokens; on en discutera plus tard dans la section sur l'architecture du transformer.

Puisque on a déjà prédit le mot suivant le token `<start>` à la première étape, on peut ignorer le premier vecteur de probabilité et se concentrer uniquement sur la seconde.


Comme le transformer est déjà entraîné, le mot avec la probabilité la plus élevée va être "aime".

On obtient donc `["I", "love", "oranges", "."]` et `["<start>", "J'", "aime"]`.

On répète ce processus jusqu'à ce que le transformer génère le token `<end>`, indiquant que la séquence générée est complète. Le transformer est entraîneé pour prédire le token <end> de manière appropriée pendant la phase d'entraînement, ce qui nous permet de déterminer quand arrêter la boucle de génération.

In [ ]:
|